In [1]:
!pip install transformers
!pip install datasets


     |████████████████████████████████| 2.0MB 6.5MB/s 
     |████████████████████████████████| 3.2MB 25.9MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5003dd2e6988f0f41dbbde1b4b8315b27de5783046e29f2b1de02038e04fcc1e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 6.3MB/s 
     |████████████████████████████████| 245kB 9.3MB/s 
     |████████████████████████████████| 112kB 9.5MB/s 


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [4]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# INPUT = "My friends are cool but they eat too many carbs."
# input = tokenizer([INPUT], max_length=500, truncation=True, padding=True, return_tensors='pt')

# #generate summary
# summary_ids = model.generate(input['input_ids'], num_beams=4, max_length=100, early_stopping=True)
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("glucose", split="train")
eval_dataset = load_dataset("glucose", split="test")

In [7]:
#tokenize a concrete item in the dictionary
#tokenized_dataset = train_dataset.map(lambda batch: tokenizer(batch["story"], truncation=True, padding=True), batched=True)

In [8]:
import torch

adjusted_training_dataset = list()
adjusted_eval_dataset = list()

for item in train_dataset:
  for dim in range(1, 11):
    if item[f"{dim}_generalNL"] != 'escaped':
      '''
        we follow the template used by the Enc-Dec model in the GLUCOSE paper
        template: #d: S*[X] generated_causal_inference
          d: dim
          S: story
          X: selected_sentence
      '''
      adjusted_training_dataset.append(f'''#{dim}: {item["story"]}*[{item["selected_sentence"]}] {item[f"{dim}_generalNL"]}''')

for item in eval_dataset:
  for dim in range(1, 11):
    if item[f"{dim}_generalNL"] != 'escaped':
      '''
        we follow the template used by the Enc-Dec model in the GLUCOSE paper
        template: #d: S*[X] generated_causal_inference
          d: dim
          S: story
          X: selected_sentence
      '''
      adjusted_eval_dataset.append(f'''#{dim}: {item["story"]}*[{item["selected_sentence"]}] {item[f"{dim}_generalNL"]}''')

train_encodings = tokenizer(adjusted_training_dataset, padding=True, truncation=True)
eval_encodings = tokenizer(adjusted_eval_dataset, padding=True, truncation=True)

In [ ]:
'''
  Define the customized dataset
'''
class AdjustedGLUCOSEDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  
  def __getitem__(self, idx):
    #get the idx-th value of each item: attention_mask, input_ids
    #self.encodings is a dict-like object
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
        return len(self.encodings.input_ids)

train_data = AdjustedGLUCOSEDataset(train_encodings)
# train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64, num_workers=2)

eval_data = AdjustedGLUCOSEDataset(eval_encodings)
# eval_loader = torch.utils.data.DataLoader(eval_data, shuffle=True, batch_size=64, num_workers=2)



In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, batch_size=64)

In [1]:
def metrics(pred):
  '''
    A callback function calculating the perplexity of generated sentence (could be extended to incoporate more metrics)
  '''
  return {
      #perlexity is just exponential of cross entropy
      'perplexity': torch.exp(pred.loss)
  }


In [2]:
from transformers import Trainer, TrainingArguments

#finetune the model using Trainer
#no need for dataloader if using Trainer
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    #per_device_train_batch_size=8,  # batch size per device during training
    #per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=600,
    do_train = True,
    do_eval = True,
    data_collator = data_collator
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = eval_data,
    compute_metrics = metrics
)

trainer.train()
trainer.save_model('./results')

ModuleNotFoundError: ignored